In [2]:
!pip install tensorflow
from google.colab import drive
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Bidirectional, LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter, filtfilt
from scipy.io import loadmat, savemat
import glob
from sklearn.utils.class_weight import compute_class_weight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 723.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00


In [3]:
# Mount Google Drive
drive.mount('/content/drive')

# Define input and output folders
input_folder = "/content/drive/MyDrive/EEG Dataset"
processed_folder = os.path.join(input_folder, "Processed_Hybrid_CNN_LSTM_BiLSTM_DNN")
model_save_folder = os.path.join(input_folder, "Models_Hybrid_CNN_LSTM_BiLSTM_DNN")
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(model_save_folder, exist_ok=True)


Mounted at /content/drive


In [5]:
# Define label mapping for 3-class classification
label_map = {"F": 0, "N": 0, "O": 1, "Z": 1, "S": 2}  # Three-class classification

# Function to load data
def load_data():
    X, y = [], []
    data_base_folder = os.path.join(input_folder, "Processed")
    data_folders = ['F', 'N', 'O', 'S', 'Z']

    for folder in data_folders:
        file_path = os.path.join(data_base_folder, f"processed_{folder}.mat")

        if not os.path.exists(file_path):
            print(f"Warning: {file_path} not found. Skipping...")
            continue

        mat_data = loadmat(file_path)
        if 'eeg_data' not in mat_data:
            print(f"Warning: 'eeg_data' key missing in {file_path}. Skipping...")
            continue

        eeg_signals = mat_data['eeg_data']
        if eeg_signals.size == 0:
            print(f"Warning: {file_path} contains no data. Skipping...")
            continue

        X.append(eeg_signals)
        y.append(np.full((eeg_signals.shape[0],), label_map[folder]))

    if not X:
        raise ValueError("No valid EEG data found. Please check the dataset.")

    X = np.vstack(X)
    y = np.concatenate(y)
    print(f"Loaded Data Shape: {X.shape}, Labels Shape: {y.shape}")
    return X, y

In [6]:
# Load dataset
X, y = load_data()

# Normalize dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape for LSTM

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define CNN-LSTM-BiLSTM-DNN Hybrid Model
def create_hybrid_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=(input_shape, 1)),

        # CNN Layers
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2, strides=2),
        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2, strides=2),

        # LSTM and BiLSTM Layers
        LSTM(64, return_sequences=True),
        Bidirectional(LSTM(32, return_sequences=False)),
        Dropout(0.4),

        # Fully Connected DNN Layers
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
        Dropout(0.4),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),

        # Output Layer
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


Loaded Data Shape: (500, 4097), Labels Shape: (500,)


In [8]:
# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []
all_y_true, all_y_pred = [], []

fold = 1
for train_index, test_index in kf.split(X):
    print(f"\nTraining Fold {fold}/10...")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_hybrid_model(input_shape=X.shape[1], num_classes=len(np.unique(y)))

    model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1
    )

    y_pred = np.argmax(model.predict(X_test), axis=1)
    accuracies.append(model.evaluate(X_test, y_test, verbose=0)[1])
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    print(f"Fold {fold} Accuracy: {accuracies[-1]:.4f}")
    fold += 1



Training Fold 1/10...
Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 12s 569ms/step - accuracy: 0.3788 - loss: 1.1343 - val_accuracy: 0.3000 - val_loss: 1.1392
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 523ms/step - accuracy: 0.6203 - loss: 1.0423 - val_accuracy: 0.6600 - val_loss: 1.0489
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 522ms/step - accuracy: 0.7208 - loss: 0.8530 - val_accuracy: 0.5000 - val_loss: 1.0347
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 517ms/step - accuracy: 0.7611 - loss: 0.7367 - val_accuracy: 0.4400 - val_loss: 1.3157
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 511ms/step - accuracy: 0.7361 - loss: 0.6760 - val_accuracy: 0.4600 - val_loss: 1.3336
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 517ms/step - accuracy: 0.7721 - loss: 0.6234 - val_accuracy: 0.4400 - val_loss: 1.3585
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 517ms/step - accuracy: 0.7430 - loss: 0.7819 - val_accuracy: 0.4400 - val_loss: 1.5797
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 510ms/step - accuracy: 0.8118 -

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 493ms/step
Fold 3 Accuracy: 0.9400

Training Fold 4/10...
Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 12s 565ms/step - accuracy: 0.2696 - loss: 1.1476 - val_accuracy: 0.3600 - val_loss: 1.1367
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 530ms/step - accuracy: 0.6803 - loss: 1.0032 - val_accuracy: 0.7200 - val_loss: 1.0288
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 521ms/step - accuracy: 0.7135 - loss: 0.8124 - val_accuracy: 0.7400 - val_loss: 0.8677
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 515ms/step - accuracy: 0.7750 - loss: 0.7182 - val_accuracy: 0.5800 - val_loss: 1.0038
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 510ms/step - accuracy: 0.7661 - loss: 0.6140 - val_accuracy: 0.5600 - val_loss: 1.0846
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 512ms/step - accuracy: 0.8320 - loss: 0.5629 - val_accuracy: 0.5200 - val_loss: 1.4227
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 525ms/step - accuracy: 0.8714 - loss: 0.4631 - val_accuracy: 0.5200 - val_loss: 1.6971
Epoch 8/10

In [10]:
# Save final trained model
model_save_path = os.path.join(model_save_folder, "hybrid_cnn_lstm_bilstm_dnn.h5")
model.save(model_save_path)
model_save_path = os.path.join(model_save_folder, "hybrid_cnn_lstm_bilstm_dnn.keras")
model.save(model_save_path)
print(f"Model saved at: {model_save_path}")


Model saved at: /content/drive/MyDrive/EEG Dataset/Models_Hybrid_CNN_LSTM_BiLSTM_DNN/hybrid_cnn_lstm_bilstm_dnn.keras


In [19]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# Load the trained model
model = load_model("/content/drive/MyDrive/EEG Dataset/Models_Hybrid_CNN_LSTM_BiLSTM_DNN/hybrid_cnn_lstm_bilstm_dnn.h5")

# Predict on the full dataset
y_probs = model.predict(X)
y_pred = np.argmax(y_probs, axis=1)

# Compute AUC-ROC Score on Full Dataset
auc_score = roc_auc_score(y, y_probs, multi_class='ovr')
print(f"\nFinal AUC-ROC Score (Full Dataset): {auc_score:.4f}")

from collections import Counter

print(f"\nFinal Test Set Class Distribution: {Counter(all_y_true)}")
print("\nFinal Classification Report:")
print(classification_report(all_y_true, all_y_pred))



16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 158ms/step

Final AUC-ROC Score (Full Dataset): 0.9967

Final Test Set Class Distribution: Counter({np.int64(0): 200, np.int64(1): 200, np.int64(2): 100})

Final Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.92      0.90       200
           1       0.92      0.95      0.94       200
           2       0.89      0.74      0.81       100

    accuracy                           0.90       500
   macro avg       0.90      0.87      0.88       500
weighted avg       0.90      0.90      0.89       500



In [13]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model(model_save_path)

# Print model summary
loaded_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 27 variables whereas the saved optimizer has 52 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_20 (Conv1D)                   │ (None, 4097, 64)            │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 4097, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_20 (MaxPooling1D)      │ (None, 2048, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_21 (Conv1D)                   │ (None, 2048, 128)           │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 2048, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_21 (MaxPooling1D)      │ (None, 1024, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 1024, 64)            │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 64)                  │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_30 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 315,528 (1.20 MB)

 Trainable params: 157,571 (615.51 KB)

 Non-trainable params: 384 (1.50 KB)

 Optimizer params: 157,573 (615.52 KB)